Script to get data from AlphaVantage from a list of symbols in a file
Unknown symbols are listed in a file and I use the AlphaVatage search function to find them

In [56]:
from importlib import reload
import pandas as pd
import numpy as np
import time
from pathlib import Path
import get_data_alphavantage
reload(get_data_alphavantage)
from get_data_alphavantage import GetDataAlphaVantage


Load n files with the list of stocks symbols

In [2]:
PATH_DATA = "../../Data/"
pd.DataFrame()

for i in range(1, 10):
    path_csv = PATH_DATA+"top_actions_paris_"+str(i)+".csv"
    if i == 1:
        df_paris = pd.read_csv(path_csv, sep=",")
    else:
        df_paris=pd.concat([df_paris,pd.read_csv(path_csv, sep=",")],sort=False)
        

df_paris.describe()

,Nom,Symbole,Dernier,Var. %,Cap. bour.,Vol.
count,429,429,429,429,429,429
unique,426,427,393,267,421,374
top,Crédit Agricole,SCHN,"0,00","0,00%","1,32B",0
freq,2,2,5,88,2,33


In [53]:
df_paris.to_hdf(PATH_DATA+"list_paris_stocks.hdf",key="Nom")
df_paris.to_csv(PATH_DATA+"list_paris_stocks.csv",sep=",")

C:\Users\hebus\AppData\Local\Temp\ipykernel_11424\3593109810.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->Index(['Nom', 'Symbole', 'Dernier', 'Var. %', 'Cap. bour.', 'Vol.'], dtype='object')]

  df_paris.to_hdf(PATH_DATA+"list_paris_stocks.hdf",key="Nom")


In [128]:
df_paris= pd.read_csv(PATH_DATA+"list_paris_stocks_comp.csv", sep=",")

In [129]:
get_data_alpha = GetDataAlphaVantage()

For each row try to request get data from alpha with symbol if not log an error have a break of 15 s between each try
Batch of n symbols because of connexion loss

In [134]:
list_err = []
group = 4
group_size = 40
limit = (1+group)*group_size
if limit > len(df_paris.index):
    limit = len(df_paris.index)

for i in range(group*group_size, limit):
    symb = df_paris.iloc[i]["Symbole"]
    path_w_csv = PATH_DATA+"dl_alpha/"+symb+"_1w.csv"
    if ".PAR" not in symb:
        symb = symb+".PAR"
    print(f"Try {symb}")
    try:
        data_1w, meta = get_data_alpha.get_stock(
            symbol=symb, interval="1w", adjusted=True)
        data_1w = pd.DataFrame(data_1w)
        data_1w.to_csv(path_or_buf=path_w_csv, sep=",")
    except ValueError:
        print(f"ERROR  requesting for {symb} ")
        list_err.append([symb, df_paris.iloc[i]["Nom"]])
    time.sleep(15)

df_list_err = pd.DataFrame(list_err, columns=["Symbol Error", "Symbol Name"])
pd.DataFrame(df_list_err).to_csv(
    path_or_buf=PATH_DATA+"dl_alpha/symbol_err_"+str(group)+".csv", sep=",")


Try NRG.PAR
Try MLCVG.PAR
Try IMDA.PAR
Try EIFF.PAR
Try VIL.PAR
Try EXE.PAR
Try PERR.PAR
Try FDE.PAR
Try UFF.PAR
Try LIN.PA.PAR
Try MMT.PAR
Try TFI.PAR
Try FII.PAR
Try MAU.PAR
Try ELEC.PAR
Try EC.PAR
Try SCHP.PAR
Try LPE.PAR
Try LI.PAR
Try UBI.PAR
Try FLY.PAR
Try SESG.PAR
Try TKO.PAR
Try SCR.PAR
Try SBDG.PAR
ERROR  requesting for SBDG.PAR 
Try AF.PAR
Try ALTA.PAR
Try POM.PAR
Try MMB.PAR
Try GDS.PAR


Concat of files with symbols in error

In [50]:
for i in range(0, 9):
    path_csv = PATH_DATA+"dl_alpha/symbol_err_"+str(i)+".csv"
    if i == 1:
        df_err_symb = pd.read_csv(path_csv, sep=",")
    else:
        df_err_symb=pd.concat([df_err_symb,pd.read_csv(path_csv, sep=",")],sort=False)
        #df_paris.append(pd.read_csv(path_csv, sep=","))

df_err_symb.to_csv(path_or_buf=PATH_DATA+"dl_alpha/symbol_err_full.csv", sep=",")
df_err_symb.describe()

,Unnamed: 0
count,168.000000
mean,11.005952
std,7.590963
min,0.000000
25%,5.000000
50%,10.000000
75%,16.000000
max,30.000000


Check if we have every symbol in the downloaded files or in the err file

In [55]:
dir_base = Path(PATH_DATA+"dl_alpha")
list_ok = []
for f in dir_base.iterdir():
    if f.is_file() and "PAR_1w.csv" in f.name:
        # f.rename(dir_base/f.name.replace(".PAR_1w","_PAR_1w"))
        list_ok.append(f.name.replace("_PAR_1w.csv", ""))
df_list_ok = pd.DataFrame(list_ok, columns=["Symbol OK"])

# df_err_symb["Symbol Clean"]=df_err_symb["Symbol Error"].replace(".PAR","")
# df_err_symb["Symbol Clean"]
# df_err_symb.replace(to_replace=".PAR",value="",inplace=True,regex=True)
df_merge=df_paris.merge(df_err_symb, on=["Symbole"],how="left",indicator=True)
 df_err_symb=pd.read_csv(PATH_DATA+"dl_alpha/symbol_err_full.csv", sep=",")
df_err_symb.head()


,Symbol OK
count,225
unique,225
top,2MX
freq,1


call the search service to try to find the symbol
still use batch

In [124]:
df_res = pd.DataFrame()
df_err = pd.DataFrame()
group = 5
group_size = 40
limit = (1+group)*group_size
if limit > len(df_err_symb.index):
    limit = len(df_err_symb.index)

for i in range(group*group_size, limit):
    try:
        symb_name = df_err_symb.iloc[i]["Symbol Name"]
        print(f"Try : {symb_name}")
        df_tmp_symb, meta = get_data_alpha.search_symbol(keyword=symb_name)
        if len(df_tmp_symb.index) > 0:
            df_res = pd.concat([df_res, df_tmp_symb.iloc[[0]]])
        else:
            df_err = pd.concat([df_err, df_err_symb.iloc[[i]]])

    except (ValueError,KeyError):
        df_err = pd.concat([df_err, df_err_symb.iloc[[i]]])
    time.sleep(15)

df_res.to_csv(path_or_buf=PATH_DATA+"dl_alpha/symbol_found_" +
              str(group)+".csv", sep=",")
df_err.to_csv(path_or_buf=PATH_DATA +
              "dl_alpha/symbol_not_found_"+str(group)+".csv", sep=",")


Try : Nicox
Try : Foncière Volta
Try : Actia Group
Try : Aroca Del Pinar
Try : Interparfums
